In [1]:
# 코랩기준 우선 설치 후 런타인 다시시작 해야됨
!pip install "scikit_learn==0.22.2.post1"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install catboost
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pandas as pd
import random
import os
import numpy as np

from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, StratifiedKFold

import warnings
warnings.filterwarnings(action='ignore')

## Data Load

In [5]:
X_train = pd.read_csv('/content/drive/MyDrive/유전체공모전/data/X_train_new.csv')
X_test = pd.read_csv('/content/drive/MyDrive/유전체공모전/data/X_test_new.csv')
y = pd.read_csv('/content/drive/MyDrive/유전체공모전/data/y_train_new.csv')
submit = pd.read_csv('/content/drive/MyDrive/유전체공모전/data/sample_submission.csv')
submit = submit.iloc[:,:1]
y_train = y['class']

In [6]:
Last_01 = pd.read_csv('/content/drive/MyDrive/유전체공모전/submissions/FINAL_2nd.csv')
Last_02 = pd.read_csv('/content/drive/MyDrive/유전체공모전/submissions/Fine_20.csv')

### Model_Baseline

In [7]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import CategoricalNB
from sklearn.multioutput import ClassifierChain
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process.gpc import GaussianProcessClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.neural_network.multilayer_perceptron import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import NearestCentroid
from sklearn.svm import NuSVC
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OutputCodeClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [8]:
class CFG:
    SEED = 20
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(CFG.SEED) # Seed 고정

In [9]:
# Models
models = [GaussianNB(),
          BernoulliNB(),
          LabelSpreading(),
          NearestCentroid(),
          LabelPropagation(),
          RidgeClassifierCV(),
          KNeighborsClassifier(),
          QuadraticDiscriminantAnalysis(),
          CalibratedClassifierCV(),
          LinearDiscriminantAnalysis(),
          AdaBoostClassifier(random_state=CFG.SEED),
          BaggingClassifier(random_state=CFG.SEED),
          DummyClassifier(random_state=CFG.SEED),
          MLPClassifier(random_state=CFG.SEED),
          RidgeClassifier(random_state=CFG.SEED),
          ExtraTreeClassifier(random_state=CFG.SEED),
          ExtraTreesClassifier(random_state=CFG.SEED),
          DecisionTreeClassifier(random_state=CFG.SEED),
          RandomForestClassifier(random_state=CFG.SEED),
          LinearSVC(multi_class="crammer_singer",random_state=CFG.SEED),
          # One-Vs-One
          SVC(random_state=CFG.SEED),
          NuSVC(random_state=CFG.SEED),
          GaussianProcessClassifier(multi_class="one_vs_one",random_state=CFG.SEED),
          # One-Vs-The-Rest
          Perceptron(random_state=CFG.SEED),
          SGDClassifier(random_state=CFG.SEED),
          GradientBoostingClassifier(random_state=CFG.SEED),
          PassiveAggressiveClassifier(random_state=CFG.SEED),
          LinearSVC(multi_class="ovr",random_state=CFG.SEED),
          GaussianProcessClassifier(multi_class="one_vs_rest",random_state=CFG.SEED),
          LogisticRegression(multi_class="ovr",solver='liblinear',random_state=CFG.SEED),
          LogisticRegressionCV(multi_class="ovr",solver='liblinear',random_state=CFG.SEED),
          CatBoostClassifier(objective='MultiClass', eval_metric="TotalF1:average=Macro", verbose=False, random_state=CFG.SEED),
          LGBMClassifier(objective='multiclass', random_state=CFG.SEED),
          XGBClassifier(objective='multi:softmax', num_class=3, random_state=CFG.SEED)]
# Models_name
models_name = list(map(str, models))
for i in range(len(models_name)):
  models_name[i] = models_name[i].split('(')[0]

In [10]:
for i in range(len(models)):
  models[i].fit(X_train,y_train)
  submit[f'{models_name[i]}'] = models[i].predict(X_test)

In [11]:
ABC = submit.iloc[:,1:].apply(pd.value_counts, axis=1).fillna(0)
ABC['rate'] = ABC.max(axis=1)/ABC.sum(axis=1)

In [12]:
submit.loc[103]

id                                                             TEST_103
GaussianNB                                                            B
BernoulliNB                                                           C
LabelSpreading                                                        C
NearestCentroid                                                       C
LabelPropagation                                                      C
RidgeClassifierCV                                                     C
KNeighborsClassifier                                                  C
QuadraticDiscriminantAnalysis                                         C
CalibratedClassifierCV                                                C
LinearDiscriminantAnalysis                                            B
AdaBoostClassifier                                                    C
BaggingClassifier                                                     C
DummyClassifier                                                 

In [13]:
Last_01['class'].compare(submit['DummyClassifier'])

,self,other
0,A,C
1,B,A
2,C,A
4,A,B
5,C,B
...,...,...
165,A,B
166,C,B
171,C,A
172,C,B
